# Templatized notebook for running CB-Geo MPM TAPIS job

## Install DesignSafe API (dapi)

In [1]:
!pip install --user --upgrade setuptools

In [2]:
!pip install --user --only-binary=:all: atomicwrites==1.4.0

In [3]:
!pip install --user jsonschema>=4.18.0

In [4]:
!pip install git+https://github.com/DesignSafe-CI/dapi.git@tapisv3 --user

  Cloning https://github.com/DesignSafe-CI/dapi.git (to revision tapisv3) to /tmp/pip-req-build-rlxjpkq5
  Running command git clone --filter=blob:none --quiet https://github.com/DesignSafe-CI/dapi.git /tmp/pip-req-build-rlxjpkq5
  Running command git checkout -b tapisv3 --track origin/tapisv3
  Switched to a new branch 'tapisv3'
  Branch 'tapisv3' set up to track remote branch 'tapisv3' from 'origin'.
  Resolved https://github.com/DesignSafe-CI/dapi.git to commit 93eb0c04a3e37805c9ee695b8bbf3ab74ad8d4aa
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [5]:
#!pip install dapi --quiet

## Modify MPM Job parameters

> Only change the cell below

In [6]:
# Default parameters for the MPM template notebook
path: str = "/MyData/mpm-benchmarks/2d/uniaxial_stress/"  # path to input file

input_file: str = "mpm.json"  # input file.json

duration: str = "00:10:00"  # Runtime duration ('HH:MM:SS')

# The parameters modified in your job will be included in the cell below.

## Imports

In [13]:
import os
import dapi
import json

In [14]:
# Authenticate 
t = dapi.auth.init()

## Configure and run MPM job

In [19]:
# Identify folder with input file in DesignSafe
input_uri      = dapi.jobs.get_ds_path_uri(t, path)
print(input_uri)

tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/


In [20]:
# Generate job info for mpm
job_info = dapi.jobs.generate_job_info(t, 'mpm', input_uri, input_file, allocation='BCS20003')
job_info['maxMinutes'] = 10
job_info['execSystemLogicalQueue'] = 'development'
print("\n---Job Info---\n\n" + json.dumps(job_info, indent=2))


---Job Info---

{
  "name": "mpm_20241001_114511",
  "appId": "mpm",
  "appVersion": "1.1.0",
  "execSystemId": "frontera",
  "maxMinutes": 10,
  "archiveOnAppError": true,
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/"
    }
  ],
  "execSystemLogicalQueue": "development",
  "nodeCount": 1,
  "coresPerNode": 1,
  "parameterSet": {
    "appArgs": [
      {
        "name": "Input Script",
        "arg": "mpm.json"
      }
    ],
    "schedulerOptions": [
      {
        "name": "TACC Allocation",
        "arg": "-A BCS20003"
      }
    ]
  }
}


## Submit and monitor job status

In [17]:
# Submit job
job = t.jobs.submitJob(**job_info)
jobUuid=job.uuid

In [21]:
# Monitor job status
dapi.jobs.get_status(t, jobUuid)

Waiting for job to start: 0it [00:00, ?it/s]
Monitoring job:   0%|                                                        | 0/40 [00:00<?, ?it/s]

	Status: RUNNING


Monitoring job:   5%|██▍                                             | 2/40 [00:30<09:31, 15.05s/it]

	Status: ARCHIVING


Monitoring job:   8%|███▌                                            | 3/40 [00:45<09:17, 15.07s/it]

	Status: FINISHED


'FINISHED'

In [22]:
# Get runtime summary
dapi.jobs.runtime_summary(t, jobUuid)


Runtime Summary
---------------
QUEUED  time: 00:00:02
RUNNING time: 00:00:37
TOTAL   time: 00:01:43
---------------


In [27]:
# Fet details from Tapis results of a job
t.jobs.getJob(jobUuid=jobUuid)


_fileInputsSpec: None
_parameterSetModel: None
appId: mpm
appVersion: 1.1.0
archiveCorrelationId: d57774ea-70d6-4873-ba2b-f7fd2a3b4f25
archiveOnAppError: True
archiveSystemDir: /home/kks32/tapis-jobs-archive/2024-10-01Z/mpm_20241001_114355-6f0c334f-5f64-463a-83b3-394921bf1eab-007
archiveSystemId: cloud.data
archiveTransactionId: 73be3f7e-825e-4569-b680-10d3c4f47acc
blockedCount: 0
cmdPrefix: None
condition: NORMAL_COMPLETION
coresPerNode: 1
created: 2024-10-01T11:44:09.196972Z
createdby: kks32
createdbyTenant: designsafe
description: mpm-1.1.0 submitted by kks32@designsafe
dtnInputCorrelationId: None
dtnInputTransactionId: None
dtnOutputCorrelationId: None
dtnOutputTransactionId: None
dtnSystemId: None
dtnSystemInputDir: None
dtnSystemOutputDir: None
dynamicExecSystem: False
ended: 2024-10-01T11:45:58.879888Z
execSystemConstraints: None
execSystemExecDir: /scratch1/05873/kks32/tapis/6f0c334f-5f64-463a-83b3-394921bf1eab-007
execSystemId: frontera
execSystemInputDir: /scratch1/05873/kks

## Archive path to view output results

In [29]:
# Get local archive path on DesignSafe
archive_path = t.jobs.getJob(jobUuid=jobUuid).archiveSystemDir
print(archive_path)

/home/kks32/tapis-jobs-archive/2024-10-01Z/mpm_20241001_114355-6f0c334f-5f64-463a-83b3-394921bf1eab-007
